# Robustness of your code

Once you have written some initial code from your prototype with good documentation and a nice modular structure (and of course carefully tracked by git) you will need to check that it is correct and robust to error (either data or user).  There are a few things you can do to make this much easier:

- I/O
- Type checking
- Error trapping
- Debugging
- Unit testing
- Continuous Integration




## I/O

One key factor that is important when creating code is how do I get the information into, and out of the code.  This can seem simple at first but there is some good practice to follow which can be helpful for avoiding error later on.

The first point is that you should never hard code any input variables.  This makes it difficult to change the behaviour of your code but also makes it impossible to run two versions simultaneously with different parameters as you would need two version of the code.

Instead you should create an initialisation file which you pass to the programme with the specific parameters required to run.  There is a standard package for this called `iniparser`

Let's see an example of how to do this:

In [ ]:
import sys
import configparser as cfg

input_file = sys.argv[1]

config = cfg.ConfigParser()
config.read(input_file)

This will allow us to pass a the name of a file containing the parameters to the programme on the command line which will be stored in `sys.argv[1]`.  We then create an instance of `configparser` and use it to read the parameter file.

In [ ]:
$ python src/main.py parameter.ini

Typically I would store parameter files in the main folder, not with the code in src.  Version controlling them is note recommended, instead put a dummy template in the `README` for new users to create their own
  
The parameter file should have the format of sections labeled by labels in square brackets, followed by the parameters in that section.  Here is an example:

In [ ]:
[cosmology]
omega_m = 0.3
omega_l = 0.7

[hyperparameters]
error_tolerance = 0.01
depth = 3

[flags]
model = local
do_polarisation = True

[output]
output_path = output/
output_name = data

It is good to separate out parameters for the physical system, the internal hyperparameters, the flags that determine operation, and the parameters that determine the environment.

These parameters are accessed via the `get` directives:

In [ ]:
omega_m = config.getfloat('cosmology', 'omega_m', fallback=0.3)
omega_l = config.getfloat('cosmology', 'omega_l', fallback=0.7)

error_tolerance = config.getfloat('hyperparameters', 'error_tolerance', fallback=0.01)
depth = config.getint('hyperparameters', 'depth', fallback=3)

model = config.get('flags', 'model', fallback='local')
do_polarisation = config.getboolean('flags', 'do_polarisation', fallback=False)

These parameters can then be used in your code.  

You should also use the parameters to create unique filenames for each run.  This avoids you accidentally  overwriting previous output, allows you to run multiple jobs simultaneously, and automatically labels your output so you know how it was generated: 

In [ ]:
output_filename = config.get('output', 'output_path')+"/"+config.get('output', 'output_name') \
    + "_"+str(omega_m)+"_"+str(omega_l)+"_"+str(error_tolerance)+"_"+str(depth)+"_"+str(model) \
    +"_"+str(do_polarisation)+".txt"

Ideally you would also create input filenames from parameters too.  This will ensure you always load the correct input data for the task to avoid wasted runs.  An example for the above is in the folder Examples/Parser

## Error trapping

It is a good idea to write you code so that when errors occurs the code reports this to you.  Python is pretty good a providing reasonable error messages so this is not as important here as in other languages, but it's good practice to make you own for situations where things could go wrong.  One simple place to check things is whether the input makes sense. So for fibonacci we could add: 

In [ ]:
def fibonacci(num):
    
    if not isinstance(num,int):
        print("non-integer input given to function fibonacci; num="+str(num))
        return 0
    
    a=0
    b=1
    for i in range(num):
        a,b = b,a+b
    return a

fibonacci(3.4)
fibonacci(3)



We could use the `typing` package, which does not affect runtime but provides hints to static analysis tools like flake8.  Its use it you just add the type after a colon.  The return type for the function is given after with an arrow:

In [ ]:
import typing

def fibonacci(num: int) -> int:
    
    if not isinstance(num,int):
        print("non-integer input given to function fibonacci; num="+str(num))
        return 0
    
    a=0
    b=1
    for i in range(num):
        a,b = b,a+b
    return a

fibonacci(3.4)
fibonacci(3)

More usefully, you can add checks for invalid input variables to avoid bad, or unexpected, data breaking routines

In [ ]:
def harmonic_mean(list):
    x = 0e0
    for item in list:
        if item==0:
            print("Harmonic mean does not exist for data containing zeros.  Returned 'None'")
            return None
        
        x+= 1e0 / item
    x = len(list)/x
    
    return x

list1 = [1,2,3,4,5,6,7,8,9]
list2 = [1,2,3,4,5,6,7,8,9,0]
print(harmonic_mean(list1))
print(harmonic_mean(list2)) 

or to trap files that can't be opened, or other things likely to produce an error, with `try`,`except` blocks:

In [ ]:
try:
    f=open("doesnotexist.txt", "r")
except IOError:
    print("File could not be opened")
except:
    print("Unexpected error")
else:
    print("File opened successfully")
    f.read()

## Debugging
If you run code that has raised an exception in jupyter or ipython you can use the magic command `%debug` to launch an interactive debugger where you can examine the code line by line.  Here are the most useful commands

| Command | Description |
|---|---|
| u | Up |
| d | Down|
| p | Print |
| q | Quit |

This is best seen in an example:

In [ ]:
def bottom_func(x):
    y = x**2
    return str(x) + " squared equals " + str(y)

def top_func(z):
    result = bottom_func(z)
    return result
        
top_func('7')

In [ ]:
%debug

This can be turned on automatically with `%pdb on` so whenever an exception is raised the debugger is launched automatically

You can also run the code interactively line by line using `%run -d` which can be more useful if your code is just wrong rather than breaking.  If you launch code with this then you can step through it with the following commands:

| Command | Description |
|---|---|
| n | Next line |
| s | Step into function|
| c | Continue to run normally |
| q | Quit |




In [ ]:
%run -d Examples/UnitTesting/src/simple.py


You can also run the debugger from the command line with:

In [ ]:
$ python3 -m pdb myscript.py

You should also check error codes that come back from routines and report on them, for instance reading a file. Better examples are checking inputs are in valid ranges for functions you've created or other things which won't cause python to crash but will mean you code gives the wrong answer, for example:

## Unit testing

The best way to debug your code is to catch them before they happen which you can do with unit testing.  The ideas is that you would set up a bunch of tests for the code then every time you do a commit to master or after doing major edits you run them to check you haven't broken anything.  For the code calculating the integral in the previous notebook you may want to to create tests that check you polynomials are OK (by checking orthonormality) or that the final integral with Gauss-Legendre quadrature is correct for large l (ie set X = $\delta_{l,200}$ and see if you get the correct answer, 0.000018285996687338485).  

Having set up these tests you can then automate them to create a test package that runs, say, before a push to a central repository.  This is standard practice in commercial development environments and if you can include them in interview test questions this will put you above the majority of applicants.

It's a good idea to get into the habit of adding them for functions, ideally before you write it.  Then you can use them to check your code does what you thought it should.  You will end up spending lots of time checking your code when you are trying to fix bugs so setting up the tests in advance can save a lot of time. Luckily in python basic ones are easy to do, you can just add it to the docstring

In [ ]:
import doctest

def function(x):
    """
    Calculate x + 2
    >>> function(5)
    7
    """
    return x+3

doctest.testmod()

This is fine for super simple tests but isn't much use once you write functions that process data rather than just a number.  There are a lot of packages available but `pytest` is the standard (which is not to say it's the best).  This runs from the terminal and looks for any functions with the name `test_somefunction` or `somefunction_test` then runs them.  These functions should contain some code to run then tests to apply to the outputs using the command `assert` which accepts any boolean argument.  If our function was:

In [ ]:
def addtwo(x):
    """
        Add 2 to x
    """
    return x+1

Then our test could look like:

In [ ]:
def test_addtwo():
    """
        Test addtwo
    """
    assert( addtwo(3)==5)

These are in the files simple.py in the directory `Examples/UnitTesting`. We can test them from the command line using

In [ ]:
%%bash
pytest Examples/UnitTesting/src/simple.py

The tests can also be held in separate files like test_simple.py:

In [ ]:
%%bash
pytest Examples/UnitTesting/test/test_simple1.py

One thing to remember is that floating point arithmetic is not exact so the test of add02 in test_simple2.py fails

In [ ]:
%%bash
pytest Examples/UnitTesting/test/test_simple2.py

To fix this pytest had a function called `approx` which by default allows a relative tolerance of 1e-6 which is mostly fine and it works on most data objects:

In [ ]:
from pytest import approx
import numpy as np
print(0.1 + 0.2 == approx(0.3))
print((0.1 + 0.2, 0.2+0.4) == approx((0.3,0.6)))
print({'a': 0.1 + 0.2, 'b': 0.2 + 0.4} == approx({'a': 0.3, 'b': 0.6}))
print(np.array([0.1, 0.2]) + np.array([0.2, 0.4]) == approx(np.array([0.3, 0.6])))
print(np.array([0.1, 0.2]) + np.array([0.2, 0.1]) == approx(0.3))

However if you are testing things near zero relative tolerances are useless.  Luckily `approx` also allows you to change the tolerances and make them relative or absolute.  If you specify both, it is true if either are satisfied.

In [ ]:
from pytest import approx
print(1.0001 == approx(1))
print(1.0001 == approx(1, rel=1e-3))
print(1.0001 == approx(1, abs=1e-3))
print(1.0001 == approx(1, rel=1e-5, abs=1e-3))

You can also specify a specific failure message with `fail` ie:

In [ ]:
from pytest import fail
def test_something():
    x = somefunc()
    if x in badthings:
        fail('A bad thing came back from somefunc()')

Note that if we specify no arguments `pytest` looks for files names `test_fimename` or `filename_test` and runs them.  In general is is best practice to put your source code and you tests in different directories as it keeps them separate and safe.  You should have one test file for each module.  Then running pytest in the test directory will check all your code or you can run on individual modules if you want.

In [ ]:
%%bash
pytest Examples/UnitTesting/

### Testing Setup

Now that we know how to build tests, how should we structure them in our repositiory?  This is up to personal preference and there is no real consensus on this other than don't put them mixed in with the code.

I prefer to have my testing directory mirror my src directory (as this is how google test framework works for C/C++/Fortran).  This causes all sorts of problems with python `import` statements which can be very confusing to fix.  I have found that the simplest approach is to make your test folder a package by adding `__init__.py` and then importing using `from src import ...`.  I.e.:

```
├── src
│   └── simple.py
└── test
    ├── __init__.py
    ├── test_simple1.py
    ├── test_simple2.py
    └── test_simple3.py
```

then:

In [ ]:
from src import simple

## Testing led development

When writing code we should try to create tests as we go, ideally **before** we even right the corresponding functions.  This way we can test to see if the fucntions we write work as we expect them to.  Generally when we create functions we make some short examples to check they are working as we expect afterwards.  Best practice is simply ot reverse this process and is known as **testing led development**. Here is an example:

Have some code like this:

In [ ]:
import numpy as np

max = 10

P = np.array([i**2 for i in range(max)])

A = 0

for i in range(max):
    for j in range(max):
        for k in range(max):
            A += P[i]*P[j]*P[k]

print(A)

We coudl speed this up by noticing that the sum is symmetric in `i,j,k` so we only need to do 1/6 of the calculation.  The catch is we have to have the correct permutation factor for each `i,j,k` combination (1 for `i=j=k`, 3 for two indicies the same and 6 for all different).  We want to create a function to do this so we first write a test that it has to pass:

In [ ]:
def test_perm_factor():
    """
        Test perm_factor
    """
    assert( perm_factor(1,1,1)==1)
    assert( perm_factor(1,1,2)==3)
    assert( perm_factor(1,2,1)==3)
    assert( perm_factor(2,1,1)==3)
    assert( perm_factor(1,2,3)==6)

Then we write the function:

In [ ]:
def perm_factor(i,j,k):
    if (i==j):
        if (j==k):
            factor = 1
        else:
            factor = 3
    else:
        if (j==k):
            factor = 3
        else:
            factor = 6
    
    return factor


test_perm_factor()

When we run the test we see that we have not defined the function to consider the case where `i==k!=j`.  Now we must correct the code and run again:

In [ ]:
def perm_factor(i,j,k):
    if (i==j):
        if (j==k):
            factor = 1
        else:
            factor = 3
    else:
        if (j==k):
            factor = 3
        elif (i==k):
            factor = 3
        else:
            factor = 6
    
    return factor

test_perm_factor()

And now we find that it works correctly and we can use it in our code:

In [ ]:
import numpy as np

max = 10

P = np.array([i**2 for i in range(max)])

A = 0

for i in range(max):
    for j in range(i+1):
        for k in range(j+1):
            A += perm_factor(i,j,k)*P[i]*P[j]*P[k]

print(A)

## Continuous Integration

Now the above is all useful but the real trick is to automate it.  Continuous Integration is a development practice where developers integrate code into a shared repository frequently, preferably several times a day. Each integration can then be verified by an automated build and automated tests.  This stops people from introducing errors which are not found until much later as the code must pass all tests before it can be accepted into the main repository.

You can set up simple versions fo this with `git hooks` locally.  Ideally we would want to run:
 - code formatter
 - linter
 - testing suite

before committing.  For automatic formatting we can use `black` (`conda install black`) which will reformat our code in-place to minimise errors from out linter, for which we use `flake8`.  Once these are complete we need to run out testing suite to confirm the code is working correctly for which we use `pytest`.  

We can implement this very simply by editing the `pre-commit.sample` file in `.git/hooks` to be:

In [ ]:
#!/bin/sh
black src test
flake8 src test
pytest



Now, whenever we commit to our repository we will automatically run these three commands.

You can automate this setup (and make it a bit cleverer) by using the package `pre-commit`.  This takes a config file you create called `.pre-commit-config.yaml` and creates a `pre-commit` hook file for you.

The `.pre-commit-config.yaml` should look like this:

In [ ]:
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.0.1
    hooks:
      - id: check-yaml
      - id: end-of-file-fixer
      - id: trailing-whitespace
      - id: mixed-line-ending
      - id: debug-statements
  - repo: https://github.com/psf/black
    rev: 23.11.0
    hooks:
      - id: black
        language_version: python3.9
      - id: black-jupyter
        language_version: python3.9
  - repo: https://github.com/pycqa/flake8
    rev: 6.0.0
    hooks:
      - id: flake8
  - repo: local
    hooks:
      - id: testing
        name: testing
        entry: pytest
        language: system
        files: ^test/ # ^ means "start with test/"
        always_run: true # run on all files, not just those staged otherwise it will not run unless you update the test file

Here we have enabled a bunch of standard checks from `pre-commit-hooks` and added a hook for both `black` and `flake8`.  This `.yaml` file is used to build a test area which will check that your code passed all the the hooks you have added. Once you have created this you tell `pre-commit` to generate the hook file with:

$ pre-commit install

This will create a `pre-commit` file in the `.git/hooks` directory. This file contains instructions on how to run the pre-commit checks which it does by building it's own version of the hooks from remotes.  You can use the local version you have installed using the `repo` tag `local`.  This is a bit cleverer than the simple coding we did above as it will only run on staged files rather than everything.  

The addition of pytest is questionable.  This will run you entire test suite every time you commit so you should only add this if your tests are very quick.  

Ideally we would only run tests on commits to `main`. Unfortunately, it is not easy to edit the `yaml` to only run some parts depending on the branch, so you can't automate this easily this way. You can however have different versions of the `.pre-commit-config.yaml` on different branches so only add the last part for `main`.  In this case you will have to manage this whenever you merge to make sure that things stay this way.

The weakness of this is that you can't make your collaborators do this, you can push the `.pre-commit-config.yaml` to the remote repo but unless they run the `pre-commit install` as part of some setup, then they will not have it.

### CI on remotes

Ideally you would have these checks run on the remote repo so no-one could push non-compliant code to it.  As this is a standard development practice there are many tools for this. Common options are: `Jenkins`, ` Travis CI`, `Circle CI`, `TeamCity`, or `Bamboo`.  You can run these locally on your machine, or via the cloud for teh remote.  Most will create multiple virtual machines so you can test on multiple versions of python simultaneously to ensure code stability and most have free options.  Setting these up can be a bit tricky so one of the easiest options is to use the CI tools built into the popular GIT hosting sites. `github`, `gitlab`, and `bitbucket` which all have the tools built in and template scripts you can use.

GitLab does this via `runners` which are scripts that specify which resources to run the tests on which makes it not so useful for our purpose (you can set-up your local machine as the `runner` but this has little advantage over just running the `pre-commit` hooks)

GitHub does allow this using their own cloud resource. We will show you how to set up a simple CI routine using the the GitHub actions to you started.  We will use the following code for our example:

In [ ]:
%%file CI_Test/basic_maths.py
"""
basic math library.
"""


def add(a, b):
    """
    add a and b
    """
    return a+b


def minus(a, b):
    """
    subtract b from a
    """
    return a-b


def multiply(a, b):
    """
    multiply a and b
    """
    return a*b


def divide(a, b):
    """
    divide a by b
    """
    return a/b



In [ ]:
%%file CI_Test/test_basic_maths.py
"""
test basic_maths.py.
"""

import basic_maths as bm


def test_add():
    """
    Test add
    """
    assert(bm.add(6, 3) == 9)


def test_minus():
    """
    Test minus
    """
    assert(bm.minus(6, 3) == 3)


def test_multiply():
    """
    Test multiply
    """
    assert(bm.multiply(6, 3) == 18)


def test_divide():
    """
    Test divide
    """
    assert(bm.divide(6, 3) == 2)



1st. - We will copy the code in these two files into a folder, create a git repository and commit them.

In [ ]:
git init
git add basic_maths.py test_basic_maths.py
git commit -m "Initial commit"

2nd. We upload it to GitHub

- login to github
- click 'repositories'
- click 'new'
- follow instructions for "push an existing repository from the command line"

3rd. We click `actions` and select `Python Package` (or `Python Package using Anaconda` / `Python Application`), then `commit new file`.  This creates a new file like the following in the new folder .github/workflows

In [ ]:
# This workflow will install Python dependencies, run tests and lint with a variety of Python versions
# For more information see: https://help.github.com/actions/language-and-framework-guides/using-python-with-github-actions

name: Python package

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build:

    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.5, 3.6, 3.7, 3.8]

    steps:
    - uses: actions/checkout@v2
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v2
      with:
        python-version: ${{ matrix.python-version }}
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install flake8 pytest
        if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
    - name: Lint with flake8
      run: |
        # stop the build if there are Python syntax errors or undefined names
        flake8 . --count --select=E9,F63,F7,F82 --show-source --statistics
        # exit-zero treats all errors as warnings. The GitHub editor is 127 chars wide
        flake8 . --count --exit-zero --max-complexity=10 --max-line-length=127 --statistics
    - name: Test with pytest
          run: pytest
          files: ^test/ # ^ means "start with test/"
          always_run: true # run on all files, not just those staged otherwise it will not run unless you update the test file


We will need to have created an enviroment file to tell GitHub how to build a replica of our local environment.  We can do this using the command (we will come back to this later) 

In [ ]:
conda env export -n enviroment_name -f environment.yml --no-builds

Now we have set up the test and it will run when we push to the remote repository.  To see this we can first do a git pull to update the local repository then edit the comment at the top of `basic_maths.py` the `add`/`commit`/`push`.  Now if we navigate to GitHub and click on actions we can see the tests running (of have run depending on how fast you are).  

At the moment there is nothing to stop us pushing rubbish that fails the test and it being added to the repo.  The script just runs them.  To stop this we need to protect it.

Go to `settings` and click `branches` and `add branch protection rule` then:
 - `Require a pull request before merging` but not require approvals
 - for `branch name pattern` write "main"
 - Select `Require status checks to pass before merging`
 - Select `Do not allow bypassing the above settings` (as the owner you could do this) 

and click `Create`.

Now go back and edit the comment at the top of `basic_maths.py` again and try to push.  You should now get the message:

In [ ]:
remote: error: GH006: Protected branch update failed for refs/heads/main.
remote: error: Changes must be made through a pull request.
To github.com:JamesFergusson/ci_test.git
 ! [remote rejected] main -> main (protected branch hook declined)
error: failed to push some refs to 'github.com:JamesFergusson/ci_test.git'

To edit the code we now have to use branches which we then merge with the main in GitHub. so create a branch and switch to it:

In [ ]:
git branch "test"
git checkout test

Now add the following function to both basic_functions.py and test_basic_functions.py:

In [ ]:
def exponentiate(a, b):
    """
    calculate a to the power of b
    """
    return a^b

In [ ]:
def test_exponentiate():
    """
    Test exponentiate
    """
    assert(bm.exponentiate(6, 3) == 216)

Now `add`/`commit` then `git push --set-upstream origin test` (it won't accept `git push` as the branch does not exist on the remote).  You should get the message:

In [ ]:
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: 
remote: Create a pull request for 'test' on GitHub by visiting:
remote:      https://github.com/JamesFergusson/CI_Test2/pull/new/test
remote: 

now go back to GitHub and click `Pull Requests` then `new pull request` and under `compare` select `test`.  This should say the tests have failed so we can't merge into main.  Go back and fix the function to:

In [ ]:
def exponentiate(a, b):
    """
    calculate a to the power of b
    """
    return a**b

Then `add`/`commit`/`push`.  Now go back to the `Pull Request` which should pass and now you can merge `test` with `main`.  Now your `main` branch is safe and can't be committed to.  without all testing passing.  This is a fairly basic CI setup which works fine for very small teams but there are a huge number of options.  The documentation is mostly OK once you have the basics so you can teach yourself but if you want to create CI for a medium size team (3+) it might be best to ask for some computer officer support.  That said, the tools for managing it are advancing rapidly so it's worth exploring what is available regularly.